# 04c - Vertex AI > Training > Custom Jobs - With Custom Container

*This notebook was developed in collaboration with [statmike](https://github.com/statmike), basing much of the core code from existing notebooks and extending to new features. Notebook author: [goodrules](https://github.com/goodrules)*

### 04 Series Overview
Where a model gets trained is where it consumes computing resources.  With Vertex AI, you have choices for configuring the computing resources available at training.  This notebook is an example of an execution environment.  When it was set up there were choices for machine type and accelerators (GPUs).  

In the `04` notebook, the model training happened directly in the notebook.  The models were then imported to Vertex AI and deployed to an endpoint for online predictions. 

In this `04a-04i` series of demonstrations, the same model is trained using managed computing resources in Vertex AI as custom training jobs.  These jobs will be demonstrated as:

-  Custom Job from a python script (`04a`), python source distribution (`04b`), and custom container (`04c`)
-  Training Pipeline that trains and saves models from a python script (`04d`), python source distribution (`04e`), and custom container (`04f`)
-  Hyperparameter Tuning Jobs from a python script (`04g`), python source distribution (`04h`), and custom container (`04i`)

### This Notebook (`04c`): An extension of `04a` that saves the Python script in a custom container. 
This notebook trains the same Tensorflow Keras model from `04` by first modifying and saving the training code as a Python module on a custom container.  While this example fits nicely in a single script, larger examples will benefit from the flexibility offered by source distributions or module storage and this notebook gives an example of making the shift.   The custom container approach will also offer faster startup times and more flexbility in areas like frequency of updates, choice of frame works, and distributed training options.

The training code is stored directly on the custom container as part of the Docker build process.  This build process uses a pre-built container as the base image and adds both packages and the training code as a Python module.  The training is conducted as a Vertex AI > Training > Custom Job that is also assigned compute resources along with the custom container for executing the training in a managed service.  This is done with the [Vertex AI Python SDK](https://googleapis.dev/python/aiplatform/latest/aiplatform.html#) using the class [`aiplatform.CustomJob()`](https://googleapis.dev/python/aiplatform/latest/aiplatform.html#google.cloud.aiplatform.CustomJob).

<img src="architectures/overview/Training.png">

### Prerequisites:
-  01 - BigQuery - Table Data Source
-  Understanding:
    -  04 - Vertex AI > Notebooks - Models Built in Notebooks with sklearn
        -  Contains a more granular review of the sklearn model training

### Overview:
- Setup Environment
- Training
    - Assemble a Python file/script for training
    - Create a Custom Container containing the Python Script
    - Store the Custom Container in Artifact Registry
    - Setup the Vertex AI > Training > Custom Job
    - Run the Vertex AI > Training > Custom Job
- Serving
    - Upload The Model to Vertex AI > Models
    - Create an Endpoint with Vertex AI > Endpoints
    - Deploy the Model to the Endpoint
- Prediction
    - Prepare a record for prediction
    - Get Predictions with Python Client
    - Get Predictions with REST
    - Get Prediction with gcloud CLI

### Resources:
- [Vertex AI Custom Container For Training](https://cloud.google.com/vertex-ai/docs/training/containers-overview)

---
## Vertex AI - Conceptual Flow

<img src="architectures/slides/04c_arch.png">

---
## Vertex AI - Workflow

<img src="architectures/slides/04c_console.png">

---
## Setup

inputs:

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'mg-ce-demos'

In [2]:
REGION = 'us-central1'
DATANAME = 'fraud'
NOTEBOOK = '04c'

# Resources
BASE_IMAGE = 'gcr.io/deeplearning-platform-release/sklearn-cpu.0-23'
DEPLOY_IMAGE = 'us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-23:latest'
TRAIN_IMAGE = 'us-docker.pkg.dev/vertex-ai/training/scikit-learn-cpu.0-23:latest'
SKLEARN_VERSION = '0.23'
TRAIN_COMPUTE = 'n1-standard-4'
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id Class splits'
C = '1.0' # based on 'best' model from notebook 04
SOLVER = 'newton-cg' # based on 'best' model from notebook 04
PENALTY = 'l2' # based on 'best' model from notebook 04

packages:

In [3]:
from google.cloud import aiplatform
from datetime import datetime

from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

clients:

In [4]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bigquery = bigquery.Client()

parameters:

In [5]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
BLOB = f"{DATANAME}/models/{NOTEBOOK}/{TIMESTAMP}/model/model.joblib"
URI = f"gs://{BUCKET}/{DATANAME}/models/{NOTEBOOK}"
DIR = f"temp/{NOTEBOOK}"

In [6]:
# Give service account roles/storage.objectAdmin permissions
# Console > IMA > Select Account <projectnumber>-compute@developer.gserviceaccount.com > edit - give role
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'mg-ce-demos-main@mg-ce-demos.iam.gserviceaccount.com'

environment:

In [7]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Training

### Assemble Python File for Training

Create the main python trainer file as `/train.py`:

In [8]:
!mkdir -p {DIR}/source/trainer

In [9]:
%%writefile {DIR}/source/trainer/train.py

# package import
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from google.cloud import bigquery
from google.cloud import storage

import argparse
import os
import sys
import joblib
import pandas as pd 
import numpy as np 
import json

# import argument to local variables
parser = argparse.ArgumentParser()
# the passed param, dest: a name for the param, default: if absent fetch this param from the OS, type: type to convert to, help: description of argument
parser.add_argument('--c_value', dest = 'c_value', default = 1.0, type = float, help = 'C value')
parser.add_argument('--penalty', dest = 'penalty', default = 'l2', type = str, help = 'Penalty term')
parser.add_argument('--solver', dest = 'solver', default = 'newton-cg', type = str, help = 'Logistic regression solver')
parser.add_argument('--var_target', dest = 'var_target', type=str)
parser.add_argument('--var_omit', dest = 'var_omit', type=str)
parser.add_argument('--project_id', dest = 'project_id', type=str)
parser.add_argument('--dataname', dest = 'dataname', type=str)
parser.add_argument('--region', dest = 'region', type=str)
parser.add_argument('--notebook', dest = 'notebook', type=str)
parser.add_argument('--bucket', dest = 'bucket', type=str)
parser.add_argument('--blob', dest = 'blob', type=str)
args = parser.parse_args()

# built in parameters for data source:
PROJECT_ID = args.project_id
DATANAME = args.dataname
REGION = args.region
NOTEBOOK = args.notebook
BUCKET = args.bucket
BLOB = args.blob
VAR_OMIT = str(args.var_omit).split(' ')

# clients
bigquery = bigquery.Client(project = PROJECT_ID)

# get schema from bigquery source
query = f"SELECT * FROM {DATANAME}.INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{DATANAME}_prepped'"
schema = bigquery.query(query).to_dataframe()

# get number of classes from bigquery source
nclasses = bigquery.query(query = f'SELECT DISTINCT {args.var_target} FROM {DATANAME}.{DATANAME}_prepped WHERE {args.var_target} is not null').to_dataframe()
nclasses = nclasses.shape[0]

# read datasets from BigQuery
train_query = f"SELECT * FROM {DATANAME}.{DATANAME}_prepped WHERE splits = 'TRAIN'"
train = bigquery.query(train_query).to_dataframe()
X_train = train.loc[:, ~train.columns.isin(VAR_OMIT)]
y_train = train[args.var_target]

# Logistic Regression
clf = LogisticRegression(C=args.c_value, penalty=args.penalty, solver=args.solver, random_state=18) #optimal hyperparameters from GridSearchCV - see notebook 04
model = clf.fit(X_train, y_train)

# Output model file
joblib.dump(model, 'model.joblib') # the model needs to be named model.joblib is order for model upload to be successful

# Upload the model to GCS
bucket = storage.Client().bucket(BUCKET)
blob = bucket.blob(BLOB)
blob.upload_from_filename('model.joblib')

Writing temp/04c/source/trainer/train.py


### Create Custom Container
- https://cloud.google.com/vertex-ai/docs/training/create-custom-container
- https://cloud.google.com/vertex-ai/docs/training/pre-built-containers
- https://cloud.google.com/vertex-ai/docs/general/deep-learning
    - https://cloud.google.com/deep-learning-containers/docs/choosing-container

#### Choose a Base Image

In [10]:
BASE_IMAGE # Defined above in Setup

'gcr.io/deeplearning-platform-release/sklearn-cpu.0-23'

#### Create the Dockerfile
A basic dockerfile thats take the base image and copies the code in and define an entrypoint - what python script to run first in this case.  Add RUN entries to pip install additional packages.

In [11]:
dockerfile = f"""
FROM {BASE_IMAGE}
WORKDIR /
## Copies the trainer code to the docker image
COPY trainer /trainer
## Sets up the entry point to invoke the trainer
ENTRYPOINT ["python", "-m", "trainer.train"]
"""
with open(f'{DIR}/source/Dockerfile', 'w') as f:
    f.write(dockerfile)

#### Setup Artifact Registry

The container will need to be stored in Artifact Registry, Container Registry or Docker Hub in order to be used by Vertex AI Training jobs.  This notebook will setup Artifact registry and push a local (to this notebook) built container to it. 

https://cloud.google.com/artifact-registry/docs/docker/store-docker-container-images#gcloud

##### Enable Artifact Registry API:
Check to see if the api is enabled, if not then enable it:

In [12]:
services = !gcloud services list --format="json" --available --filter=name:artifactregistry.googleapis.com
services = json.loads("".join(services))

if (services[0]['config']['name'] == 'artifactregistry.googleapis.com') & (services[0]['state'] == 'ENABLED'):
    print(f"Artifact Registry is Enabled for This Project: {PROJECT_ID}")
else:
    print(f"Enabeling Artifact Registry for this Project: {PROJECT_ID}")
    !gcloud services enable artifactregistry.googleapis.com

Artifact Registry is Enabled for This Project: mg-ce-demos


##### Create A Repository
Check to see if the registry is already created, if not then create it

In [13]:
check_for_repo = !gcloud artifacts repositories describe {PROJECT_ID} --location={REGION}

if check_for_repo[0].startswith('ERROR'):
    print(f'Creating a repository named {PROJECT_ID}')
    !gcloud  artifacts repositories create {PROJECT_ID} --repository-format=docker --location={REGION} --description="Vertex AI Training Custom Containers"
else:
    print(f'There is already a repository named {PROJECT_ID}')

There is already a repository named mg-ce-demos


##### Configure Local Docker to Use GCLOUD CLI

In [25]:
!gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet


{
  "credHelpers": {
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.


#### Build The Custom Container (local to notebook)

In [18]:
IMAGE_URI=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{PROJECT_ID}/{NOTEBOOK}_{DATANAME}:latest"
IMAGE_URI

'us-central1-docker.pkg.dev/mg-ce-demos/mg-ce-demos/04c_fraud:latest'

In [19]:
!docker build {DIR}/source/. -t $IMAGE_URI

[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 280B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for gcr.io/deeplearning-platform-release/skl  0.0s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 280B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for gcr.

#### Test The Custom Container (local to notebook)

In [26]:
#!docker run {IMAGE_URI} --project_id {PROJECT_ID} --dataname {DATANAME}

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/trainer/train.py", line 44, in <module>
    bigquery = bigquery.Client(project = PROJECT_ID)
  File "/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py", line 235, in __init__
    _http=_http,
  File "/opt/conda/lib/python3.7/site-packages/google/cloud/client/__init__.py", line 322, in __init__
    self, credentials=credentials, client_options=client_options, _http=_http
  File "/opt/conda/lib/python3.7/site-packages/google/cloud/client/__init__.py", line 178, in __init__
    credentials, _ = google.auth.default(scopes=scopes)
  File "/opt/conda/lib/python3.7/site-packages/google/auth/_default.py", line 616, in default
    raise exceptions.DefaultCredentialsError(_HELP_MESSAGE)
google.auth.exceptions.DefaultCredentialsError: Coul

#### Push The Custom Container To Artifact Registry

In [27]:
!docker push $IMAGE_URI

The push refers to repository [us-central1-docker.pkg.dev/mg-ce-demos/mg-ce-demos/04c_fraud]

9b6693d4: Preparing 
13fc1c23: Preparing 
034092dc: Preparing 
62ed2ffb: Preparing 
7c522b7d: Preparing 
7c6e108e: Preparing 
0fe60cdc: Preparing 
eba7f5a3: Preparing 
9ad9cb4d: Preparing 
0fe60cdc: Waiting g 
7c9cb170: Preparing 
d997fdeb: Waiting g 
7c9cb170: Waiting g 
36c1940e: Preparing 
4e5c22e4: Preparing 
4e5c22e4: Waiting g 
ca552b47: Waiting g 
bf18a086: Waiting g 
09a492af: Preparing 
bf18a086: Preparing 
834cbd4e: Preparing 
9ac077d9: Preparing 
8156a203: Layer already exists latest: digest: sha256:8b3d419541c563a12de57d88eddaf570bdcf6c528a4cb591d1c6510a47330317 size: 5335


### Setup Training Job

In [38]:
CMDARGS = [
    "--c_value=" + C,
    "--penalty=" + PENALTY,
    "--solver=" + SOLVER,
    "--var_target=" + VAR_TARGET,
    "--var_omit=" + VAR_OMIT,
    "--project_id=" + PROJECT_ID,
    "--dataname=" + DATANAME,
    "--region=" + REGION,
    "--notebook=" + NOTEBOOK,
    "--bucket=" + BUCKET,
    "--blob=" + BLOB
]

MACHINE_SPEC = {
    "machine_type": TRAIN_COMPUTE,
    "accelerator_count": 0
}

WORKER_POOL_SPEC = [
    {
        "replica_count": 1,
        "machine_spec": MACHINE_SPEC,
        "container_spec": {
            "image_uri": f"{IMAGE_URI}",
            "command": [],
            "args": CMDARGS
        }
    }
]

In [39]:
customJob = aiplatform.CustomJob(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    worker_pool_specs = WORKER_POOL_SPEC,
    base_output_dir = f"{URI}/{TIMESTAMP}",
    staging_bucket = f"{URI}/{TIMESTAMP}",
    labels = {'notebook':f'{NOTEBOOK}'}
)

### Run Training Job

In [40]:
customJob.run(
    service_account = SERVICE_ACCOUNT
)

Creating CustomJob
CustomJob created. Resource name: projects/633472233130/locations/us-central1/customJobs/231810861117734912
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/633472233130/locations/us-central1/customJobs/231810861117734912')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/231810861117734912?project=633472233130
CustomJob projects/633472233130/locations/us-central1/customJobs/231810861117734912 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/633472233130/locations/us-central1/customJobs/231810861117734912 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/633472233130/locations/us-central1/customJobs/231810861117734912 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/633472233130/locations/us-central1/customJobs/231810861117734912 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/633472233130/locations/us-central1/customJobs/23181086111773

In [41]:
customJob.display_name

'04c_fraud_20221024092349'

---
## Serving

### Upload The Model

In [42]:
modelmatch = aiplatform.Model.list(filter = f'display_name="{NOTEBOOK}_{DATANAME}"')
if modelmatch:
    print("Model Already in Registry:")
    if f'time-{TIMESTAMP}' in modelmatch[0].version_aliases:
        print("This version already loaded, no action taken.")
        model = aiplatform.Model(model_name = modelmatch[0].resource_name)
    else:
        print('Loading model as new default version.')
        model = aiplatform.Model.upload(
            display_name = f'{NOTEBOOK}_{DATANAME}',
            model_id = f'model_{NOTEBOOK}_{DATANAME}',
            parent_model =  modelmatch[0].resource_name,
            serving_container_image_uri = DEPLOY_IMAGE,
            artifact_uri = f"{URI}/{TIMESTAMP}/model",
            is_default_version = True,
            version_aliases = [f'time-{TIMESTAMP}'],
            version_description = f'time-{TIMESTAMP}',
            labels = {'notebook':f'{NOTEBOOK}'}        
        )
else:
    print('This is a new model, creating in model registry')
    model = aiplatform.Model.upload(
        display_name = f'{NOTEBOOK}_{DATANAME}',
        model_id = f'model_{NOTEBOOK}_{DATANAME}',
        serving_container_image_uri = DEPLOY_IMAGE,
        artifact_uri = f"{URI}/{TIMESTAMP}/model",
        is_default_version = True,
        version_aliases = [f'time-{TIMESTAMP}'],
        version_description = f'time-{TIMESTAMP}',
        labels = {'notebook':f'{NOTEBOOK}'}
    ) 

This is a new model, creating in model registry
Creating Model
Create Model backing LRO: projects/633472233130/locations/us-central1/models/model_04c_fraud/operations/921374594999255040
Model created. Resource name: projects/633472233130/locations/us-central1/models/972280540256272384@1
To use this Model in another session:
model = aiplatform.Model('projects/633472233130/locations/us-central1/models/972280540256272384@1')


**Note** on Version Aliases:
>Expectation is a name starting with `a-z` that can include `[a-zA-Z0-9-]`

**Retrieve a Model Resource**

[Resource](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model)
```Python
model = aiplatform.Model(model_name = f'model_{NOTEBOOK}_{DATANAME}') # retrieves default version
model = aiplatform.Model(model_name = f'model_{NOTEBOOK}_{DATANAME}@time-{TIMESTAMP}') # retrieves specific version
model = aiplatform.Model(model_name = f'model_{NOTEBOOK}_{DATANAME}', version = f'time-{TIMESTAMP}') # retrieves specific version
```

### Create An Endpoint

In [43]:
SERIES = ''.join(filter(str.isdigit, NOTEBOOK))
endpoints = aiplatform.Endpoint.list(filter = f"display_name={SERIES}_{DATANAME}")
if endpoints:
    endpoint = endpoints[0]
    print(f"Endpoint Exists: {endpoints[0].resource_name}")
else:
    endpoint = aiplatform.Endpoint.create(
        display_name = f"{SERIES}_{DATANAME}",
        labels = {'notebook':f"{SERIES}"}    
    )
    print(f"Endpoint Created: {endpoint.resource_name}")

Endpoint Exists: projects/633472233130/locations/us-central1/endpoints/8704041908730593280


In [44]:
endpoint.display_name

'04_fraud'

### Deploy Model To Endpoint

In [45]:
dmodels = endpoint.list_models()

check = 0
if dmodels:
    for dmodel in dmodels:
        if dmodel.model == model.resource_name and dmodel.model_version_id == model.version_id and dmodel.id in endpoint.traffic_split:
            print(f'This model (and version) already deployed with {endpoint.traffic_split[dmodel.id]}% of traffic')
            check = 1
    
if check == 0:
    print(f'Deploying model with 100% of traffic...')
    endpoint.deploy(
        model = model,
        deployed_model_display_name = f'{NOTEBOOK}_{DATANAME}',
        traffic_percentage = 100,
        machine_type = DEPLOY_COMPUTE,
        min_replica_count = 1,
        max_replica_count = 1
    )  

Deploying model with 100% of traffic...
Deploying Model projects/633472233130/locations/us-central1/models/972280540256272384 to Endpoint : projects/633472233130/locations/us-central1/endpoints/8704041908730593280
Deploy Endpoint model backing LRO: projects/633472233130/locations/us-central1/endpoints/8704041908730593280/operations/4856394769414225920
Endpoint model deployed. Resource name: projects/633472233130/locations/us-central1/endpoints/8704041908730593280


### Remove Deployed Models without Traffic

In [ ]:
for dmodel in endpoint.list_models():
    if dmodel.id in endpoint.traffic_split:
        print(f"Model {dmodel.display_name} has traffic = {endpoint.traffic_split[dmodel.id]}")
    else:
        endpoint.undeploy(deployed_model_id = dmodel.id)
        print(f"Undeployed {dmodel.display_name} version {dmodel.model_version_id} as it has no traffic.")

In [ ]:
endpoint.traffic_split

In [ ]:
endpoint.list_models()

---
## Prediction

### Prepare a record for prediction: instance and parameters lists

In [47]:
test_query = f"SELECT * FROM {DATANAME}.{DATANAME}_prepped WHERE splits = 'TEST' LIMIT 10"
test = bigquery.query(test_query).to_dataframe()
X_test = test.loc[:, ~test.columns.isin(str(VAR_OMIT).split(' '))]
y_test = test[VAR_TARGET]
X_test.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,15689,-0.745436,1.208740,2.418708,3.033260,-0.501836,1.497792,-0.848100,0.999515,0.376560,...,-0.149698,-0.066289,0.035959,-0.088377,-0.395340,-0.422684,0.188265,0.020712,0.045274,0.0
1,51457,-5.194546,-1.741200,2.409787,3.979815,4.745377,-0.470931,-6.630054,-1.774991,0.541608,...,-1.728441,-1.909447,0.063879,-5.298245,-0.242950,-1.611578,0.186398,0.421281,0.010040,0.0
2,152527,2.019771,0.058431,-1.041463,1.626049,0.227237,-0.196559,-0.130095,-0.015467,-0.944086,...,-0.186591,0.191799,0.445460,0.204942,0.859635,-0.411058,2.328281,-0.226044,-0.090710,0.0
3,134025,2.101359,-0.885248,-0.019706,-0.810424,-1.429294,-0.832277,-1.110466,-0.093680,-0.347834,...,0.022980,0.493235,1.524276,0.188728,0.619306,-0.262769,-0.123342,0.023263,-0.048398,0.0
4,161647,-0.484457,1.664356,1.947575,4.371354,0.613689,0.689506,0.536490,-0.021216,-2.204157,...,0.242229,0.374826,1.234508,-0.614699,0.000011,0.583962,0.825513,-0.023025,0.073964,0.0


In [48]:
instances = [X_test.to_dict(orient='split')['data'][0]]

### Get Predictions: Python Client

In [49]:
prediction = endpoint.predict(instances=instances)
prediction

Prediction(predictions=[0.0], deployed_model_id='8787283735046258688', model_version_id='1', model_resource_name='projects/633472233130/locations/us-central1/models/model_04c_fraud', explanations=None)

In [50]:
prediction.predictions[0]

0.0

In [51]:
np.argmax(prediction.predictions[0])

0

### Get Predictions: REST

In [52]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": instances}))

In [53]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    0
  ],
  "deployedModelId": "8787283735046258688",
  "model": "projects/633472233130/locations/us-central1/models/model_04c_fraud",
  "modelDisplayName": "04c_fraud",
  "modelVersionId": "1"
}


### Get Predictions: gcloud (CLI)

In [54]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[0]


---
## Remove Resources
see notebook "99 - Cleanup"